In [1]:
import pandas as pd

 <b> Manipulate analyst reports dataset (procesed sentiment analysis) </b>

In [2]:
df_rep = pd.read_csv("Sentiments.csv") 
len(df_rep)

655

In [4]:
#Create quarter column in analyst reports to merge databases
df_rep['Date'] = pd.to_datetime(df_rep['Date']) #convert to datetime
df_rep['Quarter'] = df_rep['Date'].dt.to_period("Q")

In [5]:
def clean_time(row):    
    return row.replace(hour=0, minute=0, second=0)

In [6]:
df_rep["Date"]=df_rep["Date"].apply(clean_time)

In [8]:
#Add TICS to each review
df=pd.read_csv("Tics.csv")
df_rep = pd.merge(left=df_rep, right = df, how = "left", on= 'Company')

In [9]:
#create unique identifier by row (TIC + Quarter) for merging databases
df_rep['UniqueID'] = df_rep['TIC'].map(str) +'_'+ df_rep['Quarter'].map(str)

In [10]:
#change Date format to Datetime
df_rep['Date'] = pd.to_datetime(df_rep['Date'])

pandas._libs.tslibs.timestamps.Timestamp

In [11]:
#remove reviews out of 2 year analysis period
# remove any 2018_Q4 reviews 
filter1= df_rep['Date']< '2018-09-30'
df_rep = df_rep[filter1]

# remove reviews prior to 2016_Q3
filter2= df_rep['Date'] > "2016-10-01"
df_rep = df_rep[filter2]

#check final length
len(df_rep)

586

In [12]:
from pandas.tseries.offsets import BDay
#Identify date + 1 (one day after effect)
df_rep['T1'] = df_rep['Date'] + BDay(1)

1   2018-08-16
2   2018-07-18
3   2018-08-03
4   2018-07-18
5   2018-05-17
Name: T1, dtype: datetime64[ns]

In [13]:
#Identify date + 5 (one week after effect)
df_rep['T5'] = df_rep['Date'] + BDay(5)
df_rep['T5'].head()

1   2018-08-22
2   2018-07-24
3   2018-08-09
4   2018-07-24
5   2018-05-23
Name: T5, dtype: datetime64[ns]

In [14]:
#Identify date + 30 (one month after effect)
df_rep['T30'] = df_rep['Date'] + BDay(21)
df_rep['T30'].head()

1   2018-09-13
2   2018-08-15
3   2018-08-31
4   2018-08-15
5   2018-06-14
Name: T30, dtype: datetime64[ns]

In [15]:
#Change column names for Date to become T0
df_rep.rename(columns = {'Date' : "T0"}, inplace = True)

<b> Manipulate Company data set </b>

In [16]:
df_comp = pd.read_csv("WRDS Company Data.csv", sep = ",")
df_comp.head()

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,conm,...,datacqtr,datafqtr,rdq,niq,revtq,costat,mkvaltq,prccq,prchq,prclq
0,1690,20100331,2010,2,INDL,C,D,STD,AAPL,APPLE INC,...,2010Q1,2010Q2,20100420.0,3074.0,13499.0,A,213764.4600,235.0000,237.48,190.2500
1,1690,20100630,2010,3,INDL,C,D,STD,AAPL,APPLE INC,...,2010Q2,2010Q3,20100720.0,3253.0,15700.0,A,229768.1275,251.5300,279.01,199.2500
2,1690,20100930,2010,4,INDL,C,D,STD,AAPL,APPLE INC,...,2010Q3,2010Q4,20101018.0,4308.0,20343.0,A,259906.4875,283.7500,294.73,235.5600
3,1690,20101231,2011,1,INDL,C,D,STD,AAPL,APPLE INC,...,2010Q4,2011Q1,20110118.0,6004.0,26741.0,A,297089.0496,322.5600,326.66,277.7700
4,1690,20110331,2011,2,INDL,C,D,STD,AAPL,APPLE INC,...,2011Q1,2011Q2,20110420.0,5987.0,24667.0,A,322255.8241,348.5075,364.90,324.8365


In [18]:
#create unique identifier by row (TIC + Quarter) for merging databases
df_comp['UniqueID'] = df_comp['tic'].map(str) +'_'+ df_comp['datacqtr'].map(str)

In [19]:
#remove unncesary colums
delete = ['gvkey',  'fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc','datafmt',  'curcdq','costat', 'prchq', 'prclq']
df_comp = df_comp.drop(labels=delete, axis = 1)

In [20]:
#merge analyst reports and financial data
df_final = pd.merge(left=df_rep, right = df_comp, how = "left", on= 'UniqueID')

In [22]:
#create unique time& company id for merging with stock prices
df_final['UniqueT0'] = df_final['tic'].map(str) +'_'+ df_final['T0'].map(str)
df_final['UniqueT1'] = df_final['tic'].map(str) +'_'+ df_final['T1'].map(str)
df_final['UniqueT5'] = df_final['tic'].map(str) +'_'+ df_final['T5'].map(str)
df_final['UniqueT30'] = df_final['tic'].map(str) +'_'+ df_final['T30'].map(str)

<b> Add stock data set </b>

In [25]:
df_stock = pd.read_csv("StockPrices.csv") 
print(len(df_stock))
df_stock.info()
df_stock.tail()
df_stock.columns

17220
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17220 entries, 0 to 17219
Data columns (total 8 columns):
Date         17220 non-null object
Open         17220 non-null float64
High         17220 non-null float64
Low          17220 non-null float64
Close        17220 non-null float64
Adj Close    17220 non-null float64
Volume       17220 non-null int64
Company      17220 non-null object
dtypes: float64(5), int64(1), object(2)
memory usage: 1.1+ MB


Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'Company'],
      dtype='object')

In [26]:
#change date format on stock price data
df_stock['Date'] = pd.to_datetime(df_stock['Date']) #convert to datetime

Timestamp('2016-10-03 00:00:00')

In [27]:
#remove unncesary colums
delete = ['Open', 'High', 'Low', 'Adj Close', 'Volume']
df_stock = df_stock.drop(labels=delete, axis = 1)

In [29]:
df_stock.Date = pd.to_datetime(df_stock.Date)
df_stock.Close = pd.to_numeric(df_stock.Close, 'coerce')

#df_stock = df_stock.set_index('Date')

df_stock = df_stock.groupby('Company').apply(lambda x: x.set_index('Date').resample('D').pad())

Close Company
Company Date                          
AAP     2016-06-06  150.240005     AAP
        2016-06-07  151.710007     AAP
        2016-06-08  153.520004     AAP
        2016-06-09  156.779999     AAP
        2016-06-10  153.020004     AAP

In [30]:
# move date from index to a column
df_stock = df_stock.reset_index(level=['Date'])

#reset index from company tic to generic 1,2,3...
df_stock = df_stock.reset_index(drop = True)

In [31]:
#create unique time & company id for merging with stock prices
df_stock['UniqueT'] = df_stock['Company'].map(str) +'_'+ df_stock['Date'].map(str)

In [32]:
#remove unncesary colums
delete = ['Company']
df_stock = df_stock.drop(labels=delete, axis = 1)

In [36]:
# add stock price at T0
df_T0 = pd.merge(left = df_stock, right = df_final, left_on = 'UniqueT', right_on = 'UniqueT0', how = 'right')

#Change column names for Date to become T0
df_T0.rename(columns = {'Close' : "Close_Price_T0"}, inplace = True)

#drop "date" - created column
#remove unncesary colums
delete = ['Date']
df_T0 = df_T0.drop(labels=delete, axis = 1)

In [37]:
# add stock price at T1
df_T1 = pd.merge(left = df_stock, right = df_T0, left_on = 'UniqueT', right_on = 'UniqueT1', how = 'right')

#Change column names for Date to become T0
df_T1.rename(columns = {'Close' : "Close_Price_T1"}, inplace = True)

#drop "date" - created column
#remove unncesary colums
delete = ['Date']
df_T1 = df_T1.drop(labels=delete, axis = 1)

In [38]:
# add stock price at T5
df_T5 = pd.merge(left = df_stock, right = df_T1, left_on = 'UniqueT', right_on = 'UniqueT5', how = 'right')

#Change column names for Date to become T0
df_T5.rename(columns = {'Close' : "Close_Price_T5"}, inplace = True)

#drop "date" - created column
#remove unncesary colums
delete = ['Date']
df_T5 = df_T5.drop(labels=delete, axis = 1)

In [39]:
# add stock price at T30
df_model = pd.merge(left = df_stock, right = df_T5, left_on = 'UniqueT', right_on = 'UniqueT30', how = 'right')

#Change column names for Date to become T0
df_model.rename(columns = {'Close' : "Close_Price_T30"}, inplace = True)

#drop "date" - created column
#remove unncesary colums
delete = ['Date']
df_model = df_model.drop(labels=delete, axis = 1)

In [41]:
#format date of earnings report 
df_model ['rdq'] = df_model ['rdq'].astype(str).str[:-2]

#Define function to correct datadate format
import datetime
def date1(s):
    
    e = '%Y%m%d'
    d = datetime.datetime.strptime(str(s), e)
    return d

#apply formula
df_model ['rdq'] = df_model ['rdq'].apply(date1)

In [43]:
#remove unncesary colums
delete = ['UniqueT_x', 'UniqueT_y',
        'UniqueT_x',  'UniqueT_y']
df_model = df_model.drop(labels=delete, axis = 1)

In [44]:
#create an industry variable
#define function
def industry(row):
    retail = ['AMZN' ,'WMT', 'HD',  'COST' , 'CVS', 'WBA', 'LOW', 'TGT', 'ROST',
              'TJX', 'ORLY', 'KR', 'AZO', 'AAP', 'BBY']

    if row['tic'] in retail:
        return 'RETAIL'
    else:
        return 'TECH'

#apply function
df_model['Industry'] = df_model.apply(industry, axis =1)

In [45]:
#create new variable: time until quaterly earnings announcement
df_model['TimeDelta'] = df_model['T0'] - df_model['rdq']


0   -112 days
1    -99 days
2    -97 days
3    -92 days
4    -63 days
Name: TimeDelta, dtype: timedelta64[ns]

In [46]:
#create new variable: % of stock change T0 to TX

#T1
df_model['Change_T1'] = df_model['Close_Price_T1'] - df_model['Close_Price_T0']
df_model['Per_Change_T1'] = df_model['Change_T1'] / df_model['Close_Price_T0']

#T5
df_model['Change_T5'] = df_model['Close_Price_T5'] - df_model['Close_Price_T0']
df_model['Per_Change_T5'] = df_model['Change_T5'] / df_model['Close_Price_T0']

#T30
df_model['Change_T30'] = df_model['Close_Price_T30'] - df_model['Close_Price_T0']
df_model['Per_Change_T30'] = df_model['Change_T30'] / df_model['Close_Price_T0']

In [53]:
df_model.to_csv('chkk.csv')

In [47]:
#remove unncesary columns
delete = ['Change_T1', 'Change_T5', 'Change_T30', 'Close_Price_T30',
       'Close_Price_T5', 'Close_Price_T1', 'Close_Price_T0', 'rdq',
         'UniqueT0', 'UniqueT1', 'UniqueT5', 'UniqueT30', 'UniqueID', 'T1', 'T5',
         'T30', 'datadate', 'conm', 'datacqtr']
df_model = df_model.drop(labels=delete, axis = 1)

In [48]:
# remove because of incomplete data
#delete = ['dvpspq', 'dvpsxq']
#df_model = df_model.drop(labels=delete, axis = 1)

In [49]:
# rename variables for easier modeling
#df_model.rename(columns = {'prccq' : 'Price_Close_Q', 'mkvaltq' : 'Market_Value_Q'  }, inplace = True)

In [51]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 586 entries, 0 to 585
Data columns (total 24 columns):
Company                    586 non-null object
T0                         586 non-null datetime64[ns]
Author                     586 non-null object
Finance Headline           586 non-null float64
General Headline           586 non-null float64
Finance First Paragraph    586 non-null float64
General First Paragraph    586 non-null float64
Finance Last Paragraph     586 non-null float64
General Last Paragraph     586 non-null float64
Finance Combined           586 non-null float64
General Combined           586 non-null float64
Quarter                    586 non-null object
TIC                        586 non-null object
tic                        586 non-null object
datafqtr                   586 non-null object
niq                        586 non-null float64
revtq                      586 non-null float64
mkvaltq                    577 non-null float64
prccq                      577

In [52]:
df_model.to_csv("Model Data.csv", index = None)